## **Jupyter Notebook for X-Ray Diffraction data visualization and data treatment using Profex v5.2**
- version: XRD-release-1.4 <br>
- author: William Rigaut <br>
- date: 5.06.2024  <br>

In [ ]:
#Installing required package to run the Notebook (not needed if install.sh was executed)
#!pip install ipywidgets matplotlib tqdm nodejs npm fabio openpyxl numba
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!mkdir data results
#!mkdir data/XRD results/XRD data/MOKE results/MOKE data/EDX results/EDX

#use this if you're using multiple versions of python on your computer (remplace 3.X by the version of python you installed)
#!python3.X -m ipykernel install --user

### **Interactive plot for XRD patterns, create 2D refined parameter maps using output from Profex software**

In [ ]:
#Initializing every widgets and variables needed to run XRD plots
import os, glob, time, fabio;
import matplotlib.pyplot as plt;
import numpy as np;
from ipywidgets import interactive, widgets, fixed, Layout, Button, GridBox;
from IPython.display import display;
from tqdm.notebook import tqdm;
from scipy.interpolate import griddata;
from scipy.spatial import QhullError;

#Widgets for raw and treated XRD data interactive plots
#3;SrsN3%g#3G4q
W_folderpath = widgets.Dropdown(
    options=[('Select a folder', None)]+[(elm, elm) for elm in os.listdir('./data/XRD/') if not elm.startswith('.')],
    value=None,
    description='Folderpath',
    disabled=False,
    layout=Layout(width='auto', grid_area='path')
);
W_XRD_type = widgets.Dropdown(
    options=[('Rigaku SmartLab', 1), ('ESRF Synchrotron', 2), ('Custom', 3)],
    value=1,
    description='XRD Instr.',
    disabled=False,
    layout=Layout(width='auto', grid_area='XRD')
);
W_prefix = widgets.Text(
    value='Areamap',
    placeholder='Enter prefix',
    description='Prefix',
    disabled=False,
    layout=Layout(width='auto', grid_area='prefix')
);
W_suffix = widgets.Text(
    value='ras',
    placeholder='Enter suffix',
    description='Suffix',
    disabled=False,
    layout=Layout(width='auto', grid_area='suffix')
);
W_slider_x = widgets.IntSlider(
    min=-60, max=60,
    step=5,
    description='X position',
    layout=Layout(width='auto', grid_area='X')
);
W_slider_y = widgets.IntSlider(
    min=-40, max=40,
    step=5,
    description='Y position',
    layout=Layout(width='auto', grid_area='Y')
);
W_add = widgets.ToggleButton(
    value=False,
    description='Add',
    disabled=False,
    tooltip='Save current graph to plot',
    layout=Layout(width='auto', grid_area='add')
);
W_remove = widgets.ToggleButton(
    value=False,
    description='Remove',
    disabled=True,
    tooltip='Remove current saved graph to plot',
    layout=Layout(width='auto', grid_area='remove')
);
W_remove_all = widgets.ToggleButton(
    value=False,
    description='Rem. All',
    disabled=False,
    tooltip='Remove all saved graph to plot',
    layout=Layout(width='auto', grid_area='remove_all')
);
W_add_allX = widgets.ToggleButton(
    value=False,
    description='Add all X',
    disabled=False,
    tooltip='Add all graph along the vertical line (Y constant)',
    layout=Layout(width='auto', grid_area='add_allX')
);
W_add_allY = widgets.ToggleButton(
    value=False,
    description='Add all Y',
    disabled=False,
    tooltip='Add all graph along the horizontal line (X constant)',
    layout=Layout(width='auto', grid_area='add_allY')
);
W_slider_Xrange = widgets.IntRangeSlider(
    value=[20, 70],
    min=0, max=120,
    step=1,
    description='2-Theta',
    layout=Layout(width='auto', grid_area='Xrange')
);
W_slider_Yrange = widgets.IntRangeSlider(
    value=[0, 10000],
    min=-10000, max=100000,
    step=1000,
    description='Counts',
    layout=Layout(width='auto', grid_area='Yrange')
);
W_offset = widgets.IntSlider(
    value=0,
    min=-10000, max=10000,
    step=250,
    description='Offset',
    layout=Layout(width='auto', grid_area='offset')
);
W_offset_all = widgets.Checkbox(
    value=False,
    description='Offset all',
    tooltip='When enable each saved plot will be separated by the current offset value',
    indent=False,
    layout=Layout(width='auto', grid_area='offset_all')
);
W_range_type = widgets.Dropdown(
    options=[('2Theta', 1), ('Wavevector Q', 2)],
    value=1,
    description='Range Type',
    disabled=False,
    layout=Layout(width='auto', grid_area='range_type')
);
W_enable_img = widgets.Checkbox(
    value=False,
    description='Display 2D image',
    tooltip='Display raw image from 2D detector',
    layout=Layout(width='auto', grid_area='enable_img')
);
W_bgr_type = widgets.Dropdown(
    options=[('None', 1)],
    value=1,
    description='Background',
    disabled=True,
    layout=Layout(width='auto', grid_area='bgr_type')
);
W_RR_out = widgets.Output(
    layout=Layout(width='auto', grid_area='RR_out')
);
W_map_file = widgets.Text(
    value = None,
    description='Filename',
    layout=Layout(width='auto', grid_area='map_file')
);
W_start_button = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    tooltip='Create a file containing all the refined parameters from Rietveld refinement',
    layout=Layout(width='auto', grid_area='start_button')
);
W_data_type = widgets.Dropdown(
    options=[()],
    value=None,
    description='Data to plot',
    disabled=False,
    layout=Layout(width='100%', grid_area='data_type')
);
W_plot_dim = widgets.Dropdown(
    options=[('2D map plot',1)],
    value=1,
    description='Plot type',
    disabled=True,
    layout=Layout(width='100%', grid_area='plot_dim')
);
W_data_file = widgets.Dropdown(
    options=[()],
    value=None,
    description='Datafile',
    disabled=False,
    layout=Layout(width='100%', grid_area='data_file')
);

W_tabs = widgets.Tab();
tab_names = ['Raw XRD data', 'Rietveld Refinement', 'Parameter Extraction', '2D Parameter Maps'];
W_tabs.children = [widgets.Text(description='filler1'),
                   widgets.Text(description='filler2'),
                   widgets.Text(description='filler3'),
                   widgets.Text(description='filler4')];
for i, elm in enumerate(tab_names):
    W_tabs.set_title(i, str(elm));
output = widgets.Output();

#defining all functions for the raw plots (1st tab)
def getBoundaries(folderpath, prefix, suffix):
    if folderpath is not None and len(glob.glob(f"./data/XRD/{folderpath}/{prefix}*.{suffix}")) > 0:
        if f"./data/XRD/{folderpath}/Map_Pos.dat" not in glob.glob(f"./data/XRD/{folderpath}/*.dat"):
            filepath = f"./data/XRD/{folderpath}/{prefix}*.{suffix}";
            POS_TO_FILE_LIST = []; X_LIST, Y_LIST = [], [];
    
            for filename in sorted(glob.glob(filepath)):
                with open(filename, 'r', encoding='iso-8859-1') as file:
                    for line in file:
                        if line.startswith('*MEAS_COND_AXIS_POSITION-6'):
                            x_pos = float(line.split(' ')[1].split('"')[1]);
                        if line.startswith('*MEAS_COND_AXIS_POSITION-7'):
                            y_pos = float(line.split(' ')[1].split('"')[1]);
                            break;
                file_indice = filename.split(f'{prefix}_')[1].split('.')[0];
                POS_TO_FILE_LIST.append([x_pos, y_pos, file_indice]);

            for elm in POS_TO_FILE_LIST:
                X_LIST.append(elm[0]);
                Y_LIST.append(elm[1]);
            X_LIST = sorted(list(set(X_LIST)));
            Y_LIST = sorted(list(set(Y_LIST)));
            start_x, end_x = np.min(X_LIST), np.max(X_LIST);
            start_y, end_y = np.min(Y_LIST), np.max(Y_LIST);
            step_x, step_y = (end_x - start_x)/(len(X_LIST)-1), (end_y - start_y)/(len(Y_LIST)-1);
            with open(f"./data/XRD/{folderpath}/Map_Pos.dat", 'w', encoding='iso-8859-1') as file:
                file.write(f'{start_x}\t{end_x}\t{step_x}\n{start_y}\t{end_y}\t{step_y}');
        else :
            with open(f"./data/XRD/{folderpath}/Map_Pos.dat", 'r', encoding='iso-8859-1') as file:
                start_x, end_x, step_x = file.readline().split('\t');
                start_y, end_y, step_y = file.readline().split('\t');
        start_x = int(float(start_x)); end_x = int(float(end_x)); step_x = int(float(step_x));
        start_y = int(float(start_y)); end_y = int(float(end_y)); step_y = int(float(step_y));
        W_slider_x.min, W_slider_x.max, W_slider_x.step = start_x, end_x, step_x;
        W_slider_y.min, W_slider_y.max, W_slider_y.step = start_y, end_y, step_y;
        return start_x, start_y, end_x, end_y, step_x, step_y;
    elif folderpath is not None:
        print(f'Datafiles in {folderpath}/ with corresponding names {prefix}*.{suffix} not found.')
    return None;
    
def createFilename(folderpath, prefix, suffix, slider_x, slider_y, start_x, start_y, step_x, step_y, ext):
    '''converting coordinates to indexes in Areamap00x00y.ext'''
    if folderpath is None :
        print(f"Start by selecting a folder with your data.");
        return None;
    folderpath = './data/XRD/' + folderpath;
    file_index_x = int((slider_x-start_x)/step_x + 1);
    file_index_y = int((slider_y-start_y)/step_y + 1);
    #if x or y < 10, adding an extra 0 to the name
    if file_index_x < 10 :
        file_index_x = '0'+str(file_index_x);
    if file_index_y < 10 :
        file_index_y = '0'+str(file_index_y); 
    #creating filename with different coordinates
    if ext == 'ras':
        data = f'{prefix}_0{file_index_x}0{file_index_y}.{suffix}';
        datafile = f'{folderpath}/{data}';#getting path for file
        return datafile;
    elif ext == 'dia':
        lst = f'{prefix}_0{file_index_x}0{file_index_y}.lst';
        dia = f'{prefix}_0{file_index_x}0{file_index_y}.dia';
        dia_file = f'{folderpath}/{dia}';#getting path for dia file
        lst_file = f'{folderpath}/{lst}';#getting path for lst file
        return dia_file, lst_file;

def getRangeType(range_type, wavelength):
    match range_type:
        case 1:
            factor = lambda a : a;#f(x) = x since we already have 2theta in the datafile
            current_max, current_step, current_description = 90, 1, '2Theta';
            x_label = f'2Theta (°)';
        case 2:
            pi = np.arccos(-1);
            factor = lambda a : 4*pi*np.sin(a*pi/180)/wavelength;#f(x) = 4*pi*sin(2x/2)/lambda (scattering vector Q)
            x_label = f'Wavevector Q (Å^-1)';
            current_max, current_step, current_description = 30, 1, 'Vector Q';
    W_slider_Xrange.max, W_slider_Xrange.step, W_slider_Xrange.description = current_max, current_step, current_description;
    return factor, x_label;

def getXrdType(xrd_type):
    '''defining the prefix and suffix of the datafile depending if it is Smartlab (case 1), ESRF (case 2) or custom (case 3)'''
    wavelength = 1.54060;#in Å (or 8.04 keV, CuKalpha)
    
    match xrd_type:
        case 1:
            W_prefix.value = 'Areamap'; W_suffix.value = 'ras';
            prefix = W_prefix.value; suffix = W_suffix.value;
            W_prefix.disabled = True; W_suffix.disabled = True;
        case 2:
            W_prefix.value = 'ESRF_Areamap'; W_suffix.value = 'dat';
            prefix = W_prefix.value; suffix = W_suffix.value;
            W_prefix.disabled = True; W_suffix.disabled = True;
            wavelength = 0.61992;#in Å (or 20 keV)
        case 3:
            W_prefix.disabled = False; W_suffix.disabled = False;
            prefix = W_prefix.value; suffix = W_suffix.value;
            #wavelength = 0.61992;#in Å (or 20 keV)
            wavelength = 1.54060;#in Å (or 8.04 keV, CuKalpha)
    return prefix, suffix, wavelength;

def readFromFile(datafile, slider_x, slider_y):
    x_values, y_values = [], [];
    try:
        with open(datafile, 'r', encoding='iso-8859-1') as file:
            for line in file:
                if not line.startswith('*'):#skipping the header (but contains useful info so might save some of the lines in a file?)
                    data = line.split();
                    x_values.append(float(data[0]));
                    y_values.append(float(data[1]));
    except FileNotFoundError:
        print(f'File {datafile} not found at position ({slider_x}, {slider_y})');
        return None;
    return x_values, y_values;

def display2DImg(enable_img, datafile):
    if enable_img:
        img_lst = glob.glob(f"{datafile.split('.ras')[0]}*.img");
        if len(img_lst) > 0:
            image = fabio.open(img_lst[0]);
            return image;
    return None;

def addToGraph(folderpath, slider_x, slider_y, x_values, y_values):
    global SAVE_PLOT_MULT;
    list_append = [folderpath, slider_x, slider_y, x_values, y_values];
    
    if (list_append in SAVE_PLOT_MULT):
        return 1;
    SAVE_PLOT_MULT.append(list_append);
    return 0;

def removeFromGraph(folderpath, slider_x, slider_y):
    global SAVE_PLOT_MULT;
    TMP_LIST = [sub for sub in SAVE_PLOT_MULT if (sub[0] != folderpath or sub[1] != slider_x or sub[2] != slider_y)];
    SAVE_PLOT_MULT = TMP_LIST;
    return 0;

def checkSaveState(folderpath, prefix, suffix, x_values, y_values, slider_x, slider_y,
                   add, remove, remove_all, add_allX, add_allY,
                   start_x, start_y, end_x, end_y, step_x, step_y):
    '''check the different button state to save/remove plot from saved plot list'''
    global SAVE_PLOT_MULT;
    #if reset, we empty the saved list of plots.
    if remove_all == True :
        SAVE_PLOT_MULT = [];
        return 1;
        
    if len(SAVE_PLOT_MULT) > 0:
        counter = 0;
        for line in SAVE_PLOT_MULT:
            if (line[0] == folderpath and line[1] == slider_x and line[2] == slider_y):
                W_add.value = False;
                W_remove.value = False;
                W_add.disabled = True;
                W_remove.disabled = False;
                counter = 1;
        if counter == 0 :
            W_add.value = False;
            W_remove.value = False;
            W_add.disabled = False;
            W_remove.disabled = True;
            
    if (add == True):
        addToGraph(folderpath, slider_x, slider_y, x_values, y_values);
        return 1;
    if (remove == True):
        removeFromGraph(folderpath, slider_x, slider_y);
        return 1;
    if (add_allX == True):
        x = start_x;
        while (x <= end_x):
            datafile = createFilename(folderpath, prefix, suffix, x, slider_y, start_x, start_y, step_x, step_y, 'ras');
            save_x_values, save_y_values = readFromFile(datafile, x, slider_y);
            addToGraph(folderpath, x, slider_y, save_x_values, save_y_values);
            x += step_x;
        return 1;
    if (add_allY == True):
        y = start_y;
        while (y <= end_y):
            datafile = createFilename(folderpath, prefix, suffix, slider_x, y, start_x, start_y, step_x, step_y, 'ras');
            save_x_values, save_y_values = readFromFile(datafile, slider_x, y);
            addToGraph(folderpath, slider_x, y, save_x_values, save_y_values);
            y += step_y;
        return 1;
    return 0;

def setButtonsValues(add, remove, remove_all, add_allX, add_allY, plt):
    if remove_all == True :
        W_add.disabled = False;
        W_remove.disabled = True;
        W_remove_all.value = False;
        return 0;         
    if (add == True):
        W_add.disabled = True;
        W_remove.disabled = False;
        W_add.value = False;
        return 0;
    if (remove == True):
        W_remove.disabled = True;
        W_add.disabled = False;
        W_remove.value = False;
        plt.close('all')
        return 0;
    if (add_allX == True):
        W_add_allX.value = False;
        return 0;
    if (add_allY == True):
        W_add_allY.value = False;
        return 0;
    return 0;

def getOffset(offset, offset_all):
    offset_all_value = 0;
    if offset_all:
        offset_all_value = offset;
        offset = 0;
    if offset_all_value < 0 :
        W_slider_Yrange.min, W_slider_Yrange.max = -100000, 10000;
    elif offset_all_value > 0 or not offset_all:
        W_slider_Yrange.min, W_slider_Yrange.max = -10000, 100000;
    return offset, offset_all_value;

def readFromDia(dia_file):
    LIST_VALUES = [];#list containing all the data (2theta, I_exp, I from different phases)
    try:
        with open(dia_file, 'r', encoding='iso-8859-1') as file:
            #getting informations from header
            header = next(file);
            header_spl = header.split();
            global phases;
            phases = header_spl[8:];#getting the nb of phases and phase names to add in legend plot

            #creating list for each different phases found during the refinement to plot them individually
            for i in range(4+len(phases)):
                LIST_VALUES.append([]);
            
            for line in file:
                data = line.split();
                #adding each curve to a list, LIST_VALUES contains all of the curves as [[curve1], [curve2], ect...]
                for j, LIST in enumerate(LIST_VALUES):
                    LIST.append(float(data[j]));
        #calculating the error and adding it to the plot list
        offset = 2000;#moving the error curve to -offset value
        ERROR = [LIST_VALUES[1][i] - LIST_VALUES[2][i] - offset for i, theta in enumerate(LIST_VALUES[0])];#error=calc-exp-offsetERROR);
        LIST_VALUES.append(ERROR);
        return LIST_VALUES;
    except FileNotFoundError:
        return None;

def readFromLst(lst_file, slider_x, slider_y):
    try :
        with open(lst_file, 'r', encoding='iso-8859-1') as file:
            global header_lst;
            header_lst = 'x_pos\ty_pos\t';#dynamically creating an header for the final datafile to plot refined parameter maps
            DATA_RR_OUTPUT = [slider_x, slider_y]; FIT_RR_OUTPUT = [];
        
            #reading the .lst file line by line to get lattice parameters
            current_phase = None;
            for line in file:
                if line.startswith('Rp='):
                    R_factors = line.split('  ')[-1].split(' ');
                    for elm in R_factors:
                        FIT_RR_OUTPUT.append(elm.strip());
                elif line.startswith('Local parameters and GOALs for phase'):
                    current_phase = line.split()[-1];#name of the current phase for the refined lattice parameters
                    FIT_RR_OUTPUT.append(current_phase);
                elif line.startswith('A=') or line.startswith('C=') or line.startswith('B='):
                    FIT_RR_OUTPUT.append(line.strip());
                    line_lattice=line.split('=');#the line is in this format 'C=1.234567+-0.001234' and we split it to only get the value and err
                    letter = line_lattice[0];
                    if '+-' in line :
                        lattice = line_lattice[1].split('+-');
                    else :
                        lattice = [line_lattice[1].rstrip(), '0.000000'];
                    header_lst += f'{current_phase}_{letter}\t{current_phase}_{letter}_err\t';#adding the column name to the header
                    DATA_RR_OUTPUT.append(lattice[0]);
                    DATA_RR_OUTPUT.append(lattice[1].rstrip());
        
                #extracting volume fractions now (but is actually found first in the .lst file)
                else :
                    for k, elm in enumerate(phases):
                        elmt = elm.split('=')[1];
                        #same stuff, getting volume fraction values, format can be 'QNd2Fe14B=0.456789' for example
                        if line.startswith('Q'+elmt):
                            FIT_RR_OUTPUT.append(line.strip())
                            if '+-' in line :
                                fraction = line.split('=')[1].split('+-');
                            else :
                                fraction = [line.split('=')[1].rstrip(), '0.000000'];
                            header_lst += f'Q{elmt}\tQ{elmt}_err\t';
                            DATA_RR_OUTPUT.append(fraction[0]);
                            DATA_RR_OUTPUT.append(fraction[1].rstrip());
        return DATA_RR_OUTPUT, FIT_RR_OUTPUT;
    except FileNotFoundError:
        return None;
        
def raw_plot(folderpath, XRD_type, prefix, suffix, enable_img,
             slider_x, slider_y, add, remove, remove_all, add_allX, add_allY,
             slider_Xrange, slider_Yrange, offset, offset_all, range_type, bgr_type):
    '''Plotting function checking the state of every widget and plot the corresponding graph'''
    global SAVE_PLOT_MULT;
    try :
        #fetching all the inputs from user
        prefix, suffix, wavelength = getXrdType(XRD_type);
        start_x, start_y, end_x, end_y, step_x, step_y = getBoundaries(folderpath, prefix, suffix);
        datafile = createFilename(folderpath, prefix, suffix, slider_x, slider_y, start_x, start_y, step_x, step_y, 'ras');
        factor, x_label = getRangeType(range_type, wavelength);
        x_values, y_values = readFromFile(datafile, slider_x, slider_y);
        image2D = display2DImg(enable_img, datafile);
    except TypeError:
        print("Select a folder to start");
        return 1;
    plt.close('all');
    save_state = checkSaveState(folderpath, prefix, suffix, x_values, y_values, slider_x, slider_y,
                   add, remove, remove_all, add_allX, add_allY,
                   start_x, start_y, end_x, end_y, step_x, step_y);
    offset, offset_all_value = getOffset(offset, offset_all);
    #print(datafile);
    
    #plotting the figures
    fig = plt.figure(figsize=(15, 8));
    if len(SAVE_PLOT_MULT) > 0 :
        for n, line in enumerate(SAVE_PLOT_MULT):
            if (line[0] != folderpath or line[1] != slider_x or line[2] != slider_y):
                plt.plot([factor(x) for x in line[3]], [y + (n+1)*offset_all_value for y in line[4]],
                         linewidth = 0.7, linestyle='dashed', label=f'({line[1]}, {line[2]})')
    plt.plot([factor(x) for x in x_values], [y+offset for y in y_values],
             linewidth = 0.9, color='green', label=f'({slider_x}, {slider_y})');
    
    handles, labels = plt.gca().get_legend_handles_labels();
    plt.legend(handles[::-1], labels[::-1], loc='upper right')
    plt.xlabel(x_label);
    plt.xlim(slider_Xrange);
    plt.ylim(slider_Yrange);
    plt.ylabel('Intensity (counts)');
    plt.title(f"XRD interactive plot for {folderpath}");
    plt.grid(True);
    plt.figure(dpi=400);
    plt.draw();
    if image2D is not None:
        plt.figure(figsize=(15, 8));
        plt.imshow(np.log(image2D.data[:,615:-1]));
        plt.xlim(0, 1528);
        plt.show();
    
    if save_state:
        setButtonsValues(add, remove, remove_all, add_allX, add_allY, plt);
    return 0;

def rietveld_plot(folderpath, XRD_type, slider_x, slider_y, slider_Xrange, slider_Yrange):
    try :
        #fetching all the inputs from user
        prefix, suffix, wavelength = getXrdType(XRD_type);
        start_x, start_y, end_x, end_y, step_x, step_y = getBoundaries(folderpath, prefix, suffix);
        dia_file, lst_file = createFilename(folderpath, prefix, suffix, slider_x, slider_y, start_x, start_y, step_x, step_y, 'dia');
        LIST_VALUES = readFromDia(dia_file);
        DATA_RR_OUTPUT, FIT_RR_OUTPUT = readFromLst(lst_file, slider_x, slider_y);
    except TypeError:
        print("Select a folder to start");
        return 1;
    if (LIST_VALUES is None or DATA_RR_OUTPUT is None):
        print(f'File {dia_file} not found at position ({slider_x}, {slider_y})');
        return 1;
    #setting visuals and legend for the plot
    colortab=['g', 'r', 'b', 'purple', 'black', 'grey', 'c', 'm', 'y', 'orange'];
    labeltab=['exp', 'calc', 'bgr'];
    name_phases = [phs.split('=')[1] for phs in phases];
    for phs in name_phases :
        labeltab.append(phs);
    labeltab.append('diff');
    #plotting the curves after rietveld refinement
    plt.figure(figsize=(15, 8));
    for i in range(1, len(LIST_VALUES)):
        plt.plot(LIST_VALUES[0], LIST_VALUES[i], linewidth = 0.9, color=colortab[i-1], label=labeltab[i-1]);
    plt.xlabel('2theta (deg)');
    plt.ylabel('Intensity (counts)');
    plt.xlim(slider_Xrange);
    plt.ylim(slider_Yrange);
    plt.title(f"Rietveld Refinement for {folderpath}");
    plt.legend();
    plt.grid(True);
    plt.figure(dpi=300);
    plt.show();

    W_RR_out.clear_output();
    with W_RR_out:
        global_parm = 0;
        for i, elm in enumerate(FIT_RR_OUTPUT):
            if 'Rwp' in elm:
                print(f'{elm}  {FIT_RR_OUTPUT[i+1]}\n');
            elif 'Rexp' in elm:
                continue;
            elif 'Q' in elm:
                if global_parm == 0:
                    global_parm = 1;
                    print(f'Global parameters\n**********************');
                    global_list = [y for y in FIT_RR_OUTPUT if y.startswith('Q')];
                    for elm in global_list:
                        print(f'{elm}');
                else:
                    continue;
            elif elm in name_phases:
                print(f'\n{elm} parameters\n**********************');
            else:
                print(elm);
    return 0;

def extract_param(folderpath, map_file, save_button, prefix, suffix):
    filename = W_map_file.value;
    if filename in os.listdir('./results/XRD/'):
        print(f'{filename} already exists !');
        return 0;
    if save_button:
        W_start_button.value = False;
        
        DATA_LIST = [];
        data_save_path = './results/XRD/' + filename;
        #getting all the datafile names of the folder
        lst_file_list = sorted(glob.glob(f"./data/XRD/{folderpath}/*{prefix}*.lst"));
        if len(lst_file_list) > 0:
            start_x, start_y, end_x, end_y, step_x, step_y = getBoundaries(folderpath, prefix, suffix);
            
            for lst_file in tqdm(lst_file_list):
                indexes = lst_file.split('/')[4].split('_')[1].split('.')[0];
                index_x = indexes[:3];
                index_y = indexes[3:];
                x_pos =  (int(index_x)-1)*step_x+start_x;
                y_pos =  (int(index_y)-1)*step_y+start_y;
                data_line = readFromLst(lst_file, x_pos, y_pos)[0];
                if np.abs(x_pos) + np.abs(y_pos) <= 60:
                    line_frc = "";
                    for i, elm in enumerate(data_line):
                        if elm == 'UNDEF':
                            elm = 0.0;
                        line_frc += (str(elm)+'\t');
                    #print(line_frc);
                    DATA_LIST.append(line_frc);
            
            global header_lst;
            #print(header_lst);
            with open(data_save_path, 'w') as file:
                file.write(f'{header_lst}\n');
                for line in DATA_LIST:
                    file.write(f'{line}\n');
            print(f'File created {data_save_path}');
        else:
            print("No .lst or .dia file found. Please make the refinement first !");
    else :
        print("Enter the name of your file.");
    return 0;

def plot_2D(folderpath, file_value, data_type, plot_dim):
    global header;
    try :
        file = f'./results/XRD/{W_data_file.options[file_value][0]}';
        print(file)
        with open(file, 'r') as datafile:
            tab_header = (next(datafile)).strip().split();
            if tab_header != header:
                header = tab_header;
                W_data_type.options=[(elm, i) for i, elm in enumerate(header) if i > 1 and i%2 == 0];
                if W_data_type.value!=2:
                    W_data_type.value=2;
                else:
                    W_data_type.value=None;
                    W_data_type.value=2;
                return 0;
            if data_type is None:
                return 0;
            plot_number = data_type;
            data = np.genfromtxt(datafile, delimiter='\t', skip_header=0);
    except (FileNotFoundError, TypeError):
        print("File containing refined parameters not found.")
        return 1;
    X_data = data[:, 0];
    Y_data = data[:, 1];
    Z_data = data[:, plot_number];
    Z_err_data = data[:, plot_number+1];
    
    #convert dataframes to numpy arrays
    X, Y, Z, Z_err = np.array([]), np.array([]), np.array([]), np.array([]);
    for i in range(len(X_data)):
        X = np.append(X, X_data[i]);
        Y = np.append(Y, Y_data[i]);
        Z = np.append(Z, Z_data[i]);
        Z_err = np.append(Z_err, Z_err_data[i]);
    match plot_dim:
        case 1:
            #create x-y points to be used in heatmap
            xi = np.linspace(X.min(), X.max(), 500);
            yi = np.linspace(Y.min(), Y.max(), 500);
                
            #interpolation between the points for 2D map
            try:
                zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='cubic');
                zi_err = griddata((X, Y), Z_err, (xi[None,:], yi[:,None]), method='cubic');
            except QhullError:
                print(f"Error: Data is not 2-dimensionnal, cannot create a map");
                return 1;
            # Create the contour plot
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            CS = ax1.contourf(xi, yi, zi, 100, cmap=plt.cm.rainbow, vmax=max(Z), vmin=min(Z));
            fig.colorbar(CS, ax=ax1);
            CS_err = ax2.contourf(xi, yi, zi_err, 30, cmap=plt.cm.rainbow, vmax=max(Z_err), vmin=min(Z_err));
            fig.colorbar(CS_err, ax=ax2);
            
            ax1.set_title(f'{header[plot_number]}');
            ax2.set_title(f'{header[plot_number+1]}');
                    
            plt.tight_layout();
            plt.figure(dpi=400);
            plt.show();
    return 0;
    
def redirectFromTab(folderpath, XRD_type, prefix, suffix, enable_img,
                    slider_x, slider_y, add, remove, remove_all, add_allX, add_allY,
                    slider_Xrange, slider_Yrange, offset, offset_all, range_type, bgr_type,
                    map_file, save_button, file_value, data_type, plot_dim):
    '''checking which tab is currently selected by the user'''
    #start_time = time.time();

    match W_tabs.selected_index:
        case 0 :
            raw_plot(folderpath, XRD_type, prefix, suffix, enable_img,
                     slider_x, slider_y, add, remove, remove_all, add_allX, add_allY,
                     slider_Xrange, slider_Yrange, offset, offset_all, range_type, bgr_type);
        case 1 :
            rietveld_plot(folderpath, XRD_type, slider_x, slider_y, slider_Xrange, slider_Yrange);
        case 2 :
            extract_param(folderpath, map_file, save_button, prefix, suffix);
        case 3 :
            plot_2D(folderpath, file_value, data_type, plot_dim);
    #print(f"Execution time (system) is {time.time()-start_time:.3f} s");
    return 0;

#global variables
SAVE_PLOT_MULT = [];

#interactive function needed to check an input from the user
widget = interactive(
    redirectFromTab,
    folderpath=W_folderpath,
    XRD_type=W_XRD_type,
    prefix=W_prefix,
    suffix=W_suffix,
    enable_img=W_enable_img,
    slider_x=W_slider_x,
    slider_y=W_slider_y,
    add=W_add,
    remove=W_remove,
    remove_all=W_remove_all,
    add_allX=W_add_allX,
    add_allY=W_add_allY,
    slider_Xrange=W_slider_Xrange,
    slider_Yrange=W_slider_Yrange,
    offset=W_offset,
    offset_all=W_offset_all,
    range_type=W_range_type,
    bgr_type=W_bgr_type,
    map_file=W_map_file,
    save_button=W_start_button,
    file_value=W_data_file,
    data_type=W_data_type,
    plot_dim=W_plot_dim,
    continuous_update=False
    );

#defining the different layouts of each interactive plots
raw_plot_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto auto auto auto auto',
    grid_template_columns='10% 6.66% 16.66% 16.66% 16.66% 16.66% 6.66% 10%',
    grid_template_areas='''
    "path path path X X Xrange Xrange Xrange"
    "XRD XRD XRD Y Y Yrange Yrange Yrange"
    "prefix prefix suffix add remove range_type range_type ."
    "enable_img enable_img enable_img add_allX add_allY offset offset offset_all"
    ". . . remove_all remove_all bgr_type bgr_type ."
    ". plot plot plot plot plot plot ."
    ". 2Dimg 2Dimg 2Dimg 2Dimg 2Dimg 2Dimg ."
    '''
);
rietveld_plot_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto auto',
    grid_template_columns='10% 6.66% 16.66% 16.66% 16.66% 16.66% 6.66% 10%',
    grid_template_areas='''
    "path path path X X Xrange Xrange Xrange"
    "XRD XRD XRD Y Y Yrange Yrange Yrange"
    "plot plot plot plot plot plot RR_out RR_out"
    '''
);
parameter_extraction_layout = Layout(
    width='100%',
    grid_template_rows='auto',
    grid_template_columns='20% 20% 20% 20%',
    grid_template_areas='''
    "map_file map_file start_button ."
    "plot plot plot plot"
    '''
);
map2D_layout = Layout(
    width='100%',
    grid_template_rows='auto auto',
    grid_template_columns='3% 30% 33% 30% 3%',
    grid_template_areas='''
    "plot_dim plot_dim data_type data_file data_file"
    ". plot plot plot ."
    '''
);
#defining the gridboxes
raw_gridbox = GridBox(
    children=[W_folderpath, W_XRD_type, W_prefix, W_suffix, W_enable_img,
              W_slider_x, W_slider_y, W_add, W_remove, W_remove_all, W_add_allX, W_add_allY,
              W_slider_Xrange, W_slider_Yrange, W_offset, W_offset_all, W_range_type, W_bgr_type,
              widget.children[-1]],
    layout=raw_plot_layout);
widget.children[-1].layout=Layout(width='auto', grid_area='plot');

rietveld_gridbox = GridBox(
    children=[W_folderpath, W_XRD_type, W_slider_x, W_slider_y, W_slider_Xrange, W_slider_Yrange, W_RR_out, widget.children[-1]],
    layout=rietveld_plot_layout);

parameter_extraction = GridBox(
    children=[W_map_file, W_start_button, widget.children[-1]],
    layout=parameter_extraction_layout);

map2D_gridbox = GridBox(
    children=[W_plot_dim, W_data_type, W_data_file, widget.children[-1]],
    layout=map2D_layout);

#tab menu including all the different plots
tab_children = [raw_gridbox, rietveld_gridbox, parameter_extraction, map2D_gridbox];
W_tabs.children = tab_children;
display(W_tabs, output);

def on_tab_change(change):
    with output:
        index = W_tabs.selected_index;
        #print(index)
        if index == 0:
            W_slider_Yrange.value = [0, W_slider_Yrange.value[1]];
        if index == 1:
            W_slider_Yrange.value = [-3000, W_slider_Yrange.value[1]];
            W_slider_Xrange.value = [20, 70];
            W_slider_Xrange.description = "2Theta";
        if index == 2:
            W_RR_out.clear_output();
            folderpath = W_folderpath.value;
            W_slider_Yrange.value = [-1000, W_slider_Yrange.value[1]];
            if folderpath is not None:
                W_map_file.disabled = False;
                W_map_file.value = folderpath.split('/')[-1] + '_RR_maps.dat';
            else : 
                W_map_file.disabled = True;
        if index == 3:
            global header;
            W_slider_Yrange.value = [-2000, W_slider_Yrange.value[1]];
            
            if W_map_file.value == '' or W_folderpath.value == '':
                data_save_path = sorted(glob.glob("./results/XRD/*.dat"))[0];
                W_map_file.value = data_save_path.split('/')[-1];
            elif W_folderpath.value is not None:
                data_save_path = f'./results/XRD/{W_folderpath.value}_RR_maps.dat';
            else:
                return 1;
            try:
                with open(data_save_path, 'r') as file:
                    header = next(file);
                    header = header.strip().split();
            except (FileNotFoundError, IsADirectoryError):
                W_data_file.value = 0;
            W_data_type.options=[(elm, i) for i, elm in enumerate(header) if i > 1 and i%2 == 0];
            W_data_file.options=[(file.split('/')[-1], i) for i, file in enumerate(sorted(glob.glob(f'./results/XRD/*.dat')))];
            if W_data_type.value is None:
                W_data_type.value = 2;
            for i,elm in enumerate(W_data_file.options):
                if data_save_path.split('/')[-1] == elm[0]:
                    W_data_file.value = i;
            
W_tabs.observe(on_tab_change, names='selected_index');

**@end-of-notebook**